In [10]:
from pyspark.sql.session import SparkSession
import os
path = f"{os.getcwd()}/../"

In [11]:
packages = [
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1",
    "org.apache.kafka:kafka-clients:3.2.1"
]

jars = [
    f"{path}/jars/spark-sql-kafka-0-10_2.12-3.4.1.jar",
    f"{path}/jars/kafka-clients-3.5.1.jar",
    f"{path}/jars/mysql-connector-j-8.0.31.jar",
    f"{path}/jars/commons-pool2-2.11.1.jar",
    f"{path}/jars/spark-token-provider-kafka-0-10_2.12-3.4.1.jar",
    f"{path}/jars/spark-redis-2.4.1-jar-with-dependencies.jar",
]

spark = (SparkSession
         .Builder()
         .appName(name="test_kafka")
         .master("spark://spark:7077")
         .config("spark.jars", ",".join(jars))
         .config("spark.jars.packages", ",".join(packages))
         #.config("spark.redis.host", "redis")
         #.config("spark.redis.port", "6379")
         .getOrCreate())

In [12]:
spark

In [13]:
spark.sparkContext.defaultParallelism

2

In [14]:
#spark.stop()

In [15]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, DateType, TimestampType

schema = StructType([
    StructField("id", IntegerType()),
    StructField("bme280", StructType([
        StructField("pressure", StringType()),
        StructField("temperature", StringType()),
        StructField("humidity", StringType()),
        StructField("read_datetime", StringType())
    ])),
    StructField("picow", StructType([
        StructField("local_ip", StringType()),
        StructField("temperature", DoubleType()),
        StructField("free_storage_kb", DoubleType()),
        StructField("mem_alloc_bytes", DoubleType()),
        StructField("mem_free_bytes", DoubleType()),
        StructField("cpu_freq_mhz", DoubleType())
    ]))
])

In [35]:
picow_df = (spark 
              .read
              .format("kafka") 
              .option("kafka.bootstrap.servers", "kafka:9092") 
              .option("subscribe", "iot_source")
              #.option("startingOffsets", "earliest")
              #.option("endingOffsets", "latest")
              .load())

df = (picow_df
      .where("offset > 2")
      .selectExpr("CAST(offset AS INTEGER)","CAST(timestamp AS TIMESTAMP)","CAST(key AS STRING)", "CAST(value AS STRING)", "CAST(topic AS STRING)"))

In [37]:
from pyspark.sql.functions import lit, col, from_json, regexp_replace, to_timestamp, current_timestamp, to_utc_timestamp


stg_df =  (df
            .withColumn('data', from_json(col('value'), schema))
            .selectExpr("offset as kafka_offset",
                        "timestamp as kafka_datetime",
                        "topic as kafka_topic",
                        "data.id as id",
                        "data.bme280.pressure as bme280_pressure",
                        "data.bme280.temperature as bme280_temperature",
                        "data.bme280.humidity as bme280_humidity",
                        "data.bme280.read_datetime as read_datetime",
                        "data.picow.local_ip as picow_local_ip",
                        "data.picow.temperature as picow_temperature",
                        "data.picow.mem_alloc_bytes as picow_mem_alloc_bytes",
                        "data.picow.mem_free_bytes as picow_mem_free_bytes",
                        "data.picow.cpu_freq_mhz as picow_cpu_freq_mhz",
                        "data.picow.temperature as picow_temperature",
                       )
            .drop("key","value"))

final_df = (stg_df
                .withColumn("kafka_datetime", to_utc_timestamp(col("kafka_datetime"), "UTC"))
                .withColumn("spark_process_datetime", current_timestamp())
                .withColumn("bme280_pressure",regexp_replace(col("bme280_pressure"),"hPa","").cast(DoubleType()))
                .withColumn("bme280_temperature",regexp_replace(col("bme280_temperature"),"C","").cast(DoubleType()))
                .withColumn("bme280_humidity",regexp_replace(col("bme280_humidity"),"%","").cast(DoubleType()))
                .withColumn("read_datetime", to_timestamp(col("read_datetime"), "yyyy-M-d HH:mm:s"))
                .withColumn("read_datetime", to_utc_timestamp(col("kafka_datetime"), "UTC"))
                .orderBy(col("read_datetime").desc())
           )

final_df.printSchema()
final_df.show(10,False)


root
 |-- kafka_offset: integer (nullable = true)
 |-- kafka_datetime: timestamp (nullable = true)
 |-- kafka_topic: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- bme280_pressure: double (nullable = true)
 |-- bme280_temperature: double (nullable = true)
 |-- bme280_humidity: double (nullable = true)
 |-- read_datetime: timestamp (nullable = true)
 |-- picow_local_ip: string (nullable = true)
 |-- picow_temperature: double (nullable = true)
 |-- picow_mem_alloc_bytes: double (nullable = true)
 |-- picow_mem_free_bytes: double (nullable = true)
 |-- picow_cpu_freq_mhz: double (nullable = true)
 |-- picow_temperature: double (nullable = true)
 |-- spark_process_datetime: timestamp (nullable = false)

+------------+-----------------------+-----------+--------+---------------+------------------+---------------+-----------------------+--------------+-----------------+---------------------+--------------------+------------------+-----------------+--------------------------

In [38]:
final_df.count()

2005

In [34]:
spark.conf.set("spark.redis.host", "redis")
spark.conf.set("spark.redis.port", "6379")
final_df.write.format("redis").mode("append").option("table", "iot_data").option("create_table", "true").option("key.column", "id").save()

Py4JJavaError: An error occurred while calling o489.start.
: org.apache.spark.SparkUnsupportedOperationException: Data source redis does not support streamed writing.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.streamedOperatorUnsupportedByDataSourceError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource.createSink(DataSource.scala:326)
	at org.apache.spark.sql.streaming.DataStreamWriter.createV1Sink(DataStreamWriter.scala:439)
	at org.apache.spark.sql.streaming.DataStreamWriter.startInternal(DataStreamWriter.scala:404)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:249)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke